# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [6]:
import os
import time
# Set working dir the same as the `.env` file
os.chdir('../../01_materials/labs/')
from dotenv import load_dotenv
load_dotenv() 

%reload_ext dotenv
%dotenv


In [7]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [8]:
import os
from glob import glob

# Write your code below.
# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')
print(f"PRICE_DATA directory: {price_data_dir}")

parquet_files = glob(os.path.join(price_data_dir, '*/*/*.parquet'))
print(f"Parquet files found: {parquet_files[:5]}")  # Print the first few files
datad_pf = dd.read_parquet(parquet_files,npartitions=len("ticker")).set_index("ticker")


PRICE_DATA directory: ../../05_src/data/prices/
Parquet files found: ['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [9]:
# Write your code below.
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close']
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
).assign(    
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))


C:\Users\iryna\AppData\Local\Temp\ipykernel_18688\3011444353.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [11]:
datad_pf=datad_pf.repartition(npartitions=2)
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532223,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,51.502148,NaN,NaN,8.270744
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206841,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,47.567955,51.502148,-0.076389,2.950645
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712803,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,44.617310,47.567955,-0.062030,4.425964
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276852,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,42.918453,44.617310,-0.038076,2.771820
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299931,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,46.494991,42.918453,0.083333,4.962444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-06-27,174.490005,181.660004,174.259995,176.460007,176.460007,4164700,Health Care,Pharmaceuticals,2024,170.899994,176.460007,170.899994,0.032534,7.400009
ZTS,2024-06-28,176.460007,177.070007,173.029999,173.360001,173.360001,4189600,Health Care,Pharmaceuticals,2024,176.460007,173.360001,176.460007,-0.017568,4.040009
ZTS,2024-07-01,173.259995,175.460007,172.000000,172.580002,172.580002,1589800,Health Care,Pharmaceuticals,2024,173.360001,172.580002,173.360001,-0.004499,3.460007


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [10]:
# Write your code below.
datad_pf=datad_pf.repartition(npartitions=2)
# Convert to pandas DataFrame
pd_feat = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()

# Display the result
print(pd_feat.tail())

             Date        Open        High         Low       Close   Adj Close  \
ticker                                                                          
ZTS    2024-06-27  174.490005  181.660004  174.259995  176.460007  176.460007   
ZTS    2024-06-28  176.460007  177.070007  173.029999  173.360001  173.360001   
ZTS    2024-07-01  173.259995  175.460007  172.000000  172.580002  172.580002   
ZTS    2024-07-02  172.490005  175.990005  172.490005  174.809998  174.809998   
ZTS    2024-07-03  175.440002  177.669998  175.259995  175.600006  175.600006   

         Volume       sector        subsector  year  Close_lag_1   Adj_Close  \
ticker                                                                         
ZTS     4164700  Health Care  Pharmaceuticals  2024   170.899994  176.460007   
ZTS     4189600  Health Care  Pharmaceuticals  2024   176.460007  173.360001   
ZTS     1589800  Health Care  Pharmaceuticals  2024   173.360001  172.580002   
ZTS     2454200  Health Care  Ph

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.